첫 샘플의 베스트

"reindeer mistletoe elf gingerbread family advent scrooge chimney fireplace ornament"

# Genetic Algorithm

## Imports

In [1]:
import gc ## for memory
import os
import copy
from math import exp
from collections import Counter
from typing import List, Optional, Union

import numpy as np
import itertools
import pandas as pd
import transformers
import torch

from kaggle_evaluate import PerplexityCalculator

In [2]:
df_sample = pd.read_csv("sample_submission.csv")

In [3]:
df_sample.text.str.split().map(lambda x : len(set(x)))

0    10
1    20
2    20
3    29
4    50
5    89
Name: text, dtype: int64

## Setting

In [4]:
evaluatr = PerplexityCalculator("google/gemma-2-9b")

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

## Algorithm

`-` Target

In [5]:
samples = df_sample.text[1:6].to_list()
samples

['advent chimney elf family fireplace gingerbread mistletoe ornament reindeer scrooge walk give jump drive bake the sleep night laugh and',
 'yuletide decorations gifts cheer holiday carol magi nutcracker polar grinch sleigh chimney workshop stocking ornament holly jingle beard naughty nice',
 'yuletide decorations gifts cheer holiday carol magi nutcracker polar grinch sleigh chimney workshop stocking ornament holly jingle beard naughty nice sing cheer and of the is eat visit relax unwrap',
 'hohoho candle poinsettia snowglobe peppermint eggnog fruitcake chocolate candy puzzle game doll toy workshop wonder believe dream hope peace joy merry season greeting card wrapping paper bow fireplace night cookie milk star wish wreath angel the to of and in that have it not with as you from we kaggle',
 'advent chimney elf family fireplace gingerbread mistletoe ornament reindeer scrooge walk give jump drive bake the sleep night laugh and yuletide decorations gifts cheer holiday carol magi nutcrac

`-` Code

In [6]:
## initiate
initial_times = 1000
genomes = ["" for _ in range(initial_times)]

for i in range(initial_times) :
    genomes[i] = np.random.permutation(samples[0].split())
    
genomes = np.array(genomes)

## selection
perplexities = np.array(evaluatr.get_perplexity([" ".join(genome) for genome in genomes], batch_size = 1024))
per_sum = sum(1/perplexities)
proba = 1/perplexities/per_sum

parents_indx = np.random.choice([i for i in range(initial_times)], p = proba, size = 10, replace = False)

## crossover


In [7]:
parents_proba = perplexities[parents_indx]
per_sum = sum(parents_proba)
proba = parents_proba/per_sum

cross_area = [[int(len(s.split())/4), int(len(s.split())/2), len(s.split())] for s in samples]

* 해당 영역에 공통으로 포함된 토큰을 배제 + 중복되는 토큰 처리
* 해당 영역의 반대쪽 부모에게서만 있는 토큰을 매핑하여 지정 + 중복되는 토큰 처리
* 공란에 나머지 토큰들을 순서대로 삽입

중복되는 토큰들은 뒤에 숫자를 매겨서(더미 토큰) 유니크하게 만든 다음에 따로 관리하는 게 좋지 않을까? -> 최적화 이슈는 있지만 딱히 상관없고, 구현이 쉬움

In [131]:
def crossover(parents_indx : List[int], genomes : List[str], sample_num : int, dupl = False) -> int :
    pair_parents = list(itertools.combinations(parents_indx, 2)) ## combination of parents
    
    for pair in pair_parents :
        parents = [genomes[pair[0]], genomes[pair[1]]]
        childs = []
        
        ## dealing with duplication
        if dupl :
            for i in range(2) :
                for t in set(parents[i].split()) :
                    times = sum(np.array(parents[i].split()) == t)
                    rep = 1
                    
                    if times > 1 :
                        for j, k in enumerate(parents[i].split()) :
                            if k == t :
                                parents[i][j] = f"{t}{rep}"
                                rep += 1
        
        # PMX : i is main / 1-i is sub
        for i in range(2) :
            for j in range(100) :
                if np.random.random() < 0.2 :
                    width = np.random.randint(cross_area[sample_num][0], cross_area[sample_num][2])
                    
                else :
                    width = np.random.randint(cross_area[sample_num][0], cross_area[sample_num][1])
                    
                start = np.random.randint(0, cross_area[sample_num][2]-width)
                child = [" " for _ in range(cross_area[sample_num][2])]
                child[start:start+width] = parents[i][start:start+width]  ## child에 교차영역 복사
                
                ## mapping
                mapping_set = set(parents[1-i][start:start+width]) - set(parents[i][start:start+width])
                
                for t in mapping_set :
                    sub = np.where(np.array(parents[1-i]) == t)
                    
                    for k in range(cross_area[sample_num][2]) :
                        sub = np.where(np.array(parents[1-i]) == parents[i][sub])
                        
                        if (sub[0] < start) | (sub[0] > start+width) :
                            child[int(sub[0])] = t
                            break
                
                ## remain set
                current_set = [t for t in child if t != " "]
                remain_set = [t for t in parents[1-i] if t not in current_set]
                
                empty_indx = [i for i, t in enumerate(child) if t == " "]
                
                for k, ind in enumerate(empty_indx) :
                    child[ind] = remain_set[k]

                ## text transformation
                if dupl :
                    for j, t in enumerate(child) :
                        try :
                            int(t[-1])
                            child[i][j] = t[:-1]
                        except :
                            pass
                
                childs.append(child)
    
    return childs

In [132]:
childs = crossover(parents_indx=parents_indx, genomes = genomes, sample_num = 0, dupl = False)

/tmp/ipykernel_37794/2479703124.py:44: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  child[int(sub[0])] = t


In [166]:
mutation_area = [5, 5, 7, 10, 15]

In [203]:
def mutation(childs : List[str], sample_num : int) -> List[str] :
    childs = np.array(childs)
    
    for i in range(len(childs)) :
        for _ in range(mutation_area[sample_num]) :
            dice = np.random.randint(0, 4) ## roll a dice
            
            ## swap
            if dice == 0 :                
                swap_area = np.random.choice([i for i in range(cross_area[sample_num][2])], size = 2, replace = False)
                childs[i][swap_area[0]], childs[i][swap_area[1]] = childs[i][swap_area[1]], childs[i][swap_area[0]]
                
            ## move
            elif dice == 1 :
                moving_indx = np.random.randint(0, cross_area[sample_num][2])
                mover = childs[i][moving_indx]
                
                moving_area = np.random.randint(0, cross_area[sample_num][2]-1)
                
                ## trick
                tmp = list(childs[i])
                del tmp[moving_indx]
                tmp.insert(moving_area, mover)
                
                childs[i] = np.array(tmp)
                
            ## inverse
            elif dice == 2 :
                width = np.random.randint(3, 6)
                start = np.random.randint(0, cross_area[sample_num][2]-width)
                childs[i][start:start+width] = childs[i][start:start+width][::-1]
                
            ## scramble
            elif dice == 3 :
                swap_size = np.random.randint(3, 6)
                swap_area = np.random.choice([i for i in range(cross_area[sample_num][2])], size = swap_size, replace = False)
                
                childs[i][swap_area] = np.random.permutation(np.array(childs[i])[swap_area])
        
    return [" ".join(child) for child in childs]

In [205]:
mutation(childs, sample_num = 0)

## Submission

In [ ]:
df_submission = pd.DataFrame({"id":[0,1,2,3,4,5], "text":[" ".join(w) for w in best_word]})
df_submission.iloc[0, 1] = "reindeer mistletoe elf gingerbread family advent scrooge chimney fireplace ornament"
df_submission.to_csv("beam.csv")

os.system("kaggle competitions submit -c santa-2024 -f beam.csv -m .")